# Иcточники данных

- Внешние источники
  - b1: `b1_analytics.pkl`
  - kamaflow: `kamaflow_researches.pkl`
- Внутренние источники
  - Отраслевой хаб: данные выложены в sberdisk

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```
Мы хотим получить такой, при этом часть полей можент быть пустой:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "summary": "Саммари на основе pdfs",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here",
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

Для дальнейнего анализа нужно только следующие поля: date, sber_tags, summary, pdfs['full_text'], pdfs['summary']:
```json
{
    "date": "2023-10-15",
    "sber_tags": ["Теги", "из", "сбера", "отрасль", "из", "kksb_clinet_profile"],
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
            "summary": "Саммари из full_text",
        }
    ]
}
```

In [20]:
import pickle


with open('b1_analytics.pkl', 'rb') as f:
    b1_analytics = pickle.load(f)

with open('kamaflow_researches.pkl', 'rb') as f:
    kamaflow_researches = pickle.load(f)

# with open('/Users/22926900/Desktop/consult_plan_v3/industry_analytics_parser/notebooks/industry_hub_analytics_update.pkl', 'rb') as f:
#     industry_hub_analytics = pickle.load(f)

docs = b1_analytics + kamaflow_researches #+ industry_hub_analytics

In [21]:
import re
import json

def extract_json_blocks(text):
    """
    Извлекает содержимое всех блоков ```json ... ``` из текста.
    
    Аргументы:
        text (str): Исходный текст, содержащий блоки с JSON.
        
    Возвращает:
        str: Содержимое всех JSON-блоков, объединенное через пробел.
             Если блоков не найдено, возвращает пустую строку.
    """
    pattern = r'```json(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    if not matches:
        return ""
    
    # Объединяем все найденные блоки через пробел и убираем лишние пробелы
    result = ' '.join(match.strip() for match in matches)
    return result


def clean_json(broken_json: str) -> str:
    """
    1) Убирает висячие запятые перед ']' или '}'.
    2) Экранирует внутренние кавычки внутри строковых значений.
    Возвращает исправленный JSON в виде строки.
    """
    # 1) Удаляем запятые перед ] или }
    no_trailing = re.sub(r',\s*(?=[}\]])', '', broken_json)

    # 2) Функция для экранирования внутренних кавычек
    def escape_inner_quotes(s: str) -> str:
        res = []
        inside = False
        i = 0
        while i < len(s):
            c = s[i]
            if c == '"' and (i == 0 or s[i-1] != '\\'):
                if not inside:
                    inside = True
                    res.append(c)
                else:
                    # lookahead: если за кавычкой идёт :, , , ] или }, считаем её закрывающей
                    j = i + 1
                    while j < len(s) and s[j].isspace():
                        j += 1
                    if j < len(s) and s[j] in [':', ',', ']', '}']:
                        inside = False
                        res.append(c)
                    else:
                        # внутренняя кавычка — экранируем
                        res.append('\\"')
                i += 1
            elif c == '\\' and inside:
                # сохраняем существующие escape-последовательности без изменения
                if i + 1 < len(s):
                    res.append(c)
                    res.append(s[i+1])
                    i += 2
                else:
                    res.append(c)
                    i += 1
            else:
                res.append(c)
                i += 1
        return ''.join(res)

    cleaned = escape_inner_quotes(no_trailing)
    return cleaned


def fix_json(json_str: str) -> str:
    try:
        json.loads(json_str)
    except:
        try:
            json.loads(extract_json_blocks(json_str))
        except:
            try:
                json_str = clean_json(json_str)
                json.loads(json_str)
            except:
                import dirtyjson
                print(json_str)
                json_str = json.dumps(dict(dirtyjson.loads(json_str)), ensure_ascii=False, indent=4)
    return json_str

## Добавляем саммари

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
import pandas as pd
import tempfile
from langchain_core.prompts import SystemMessagePromptTemplate
from tqdm import tqdm
import json


llm = ChatOpenAI(model_name="gpt-4.1-nano")

prompt = """
Ты — стратегический консультант. На основе отраслевых данных, новостей и исследований тебе нужно выявить 1–3 ключевых тренда, подтверждённых фактами, и представить их в виде валидного JSON по заданной схеме.

## Твоя задача:
1. Найди до трёх трендов из текста.
2. Каждый тренд должен быть явно выражен, подтверждён конкретным фактом и проанализирован.
3. Анализ включает причины, последствия, влияние на рынок и прогноз развития (3–7 предложений).
4. Пиши только на русском языке.

## Формат вывода (JSON):
```json
[
  {{
    "тренд": "Краткое утверждение, отражающее суть тренда (например: \\"рост спроса на экологичные товары\\")",
    "факт": "Цифры, статистика или цитата из текста, подтверждающая тренд (например: \\"73% потребителей готовы платить больше за экологичную упаковку\\")",
    "анализ": "Связный аналитический текст из 3–7 предложений, раскрывающий причины, влияние, последствия и прогноз тренда"
  }}
]
```

## Требования к JSON:
- Используй только двойные кавычки (")
- После каждого поля ставь запятую, кроме последнего
- Не используй комментарии
- Все кавычки внутри строк экранируй: \\" 
- Экранируй спецсимволы: \\\\ — обратная косая, \\n — перенос строки, \\t — табуляция
- При отсутствии трендов верни: `[]`
- Все кавычки внутри строк обязательно экранируй: \\" — иначе JSON будет невалидным
- JSON должен быть полностью валидным

## Входные данные:
- Название: {name}
- Теги: {tags} 
- Краткое содержание: {additional_text}
- Текст документа:
```
{text_document}
```
"""

messages = ChatPromptTemplate.from_messages([
    ('system', prompt),
    ('human', "Выведи итоговый JSON"),
])


chain = (
    messages
    | llm 
    | StrOutputParser()
)

In [ ]:
system_prompt = '''
Ты — аналитик и JSON-корректор. На входе — повреждённый JSON со списком трендов, содержащий синтаксические ошибки (неэкранированные кавычки, лишние запятые, некорректные структуры, обрывки текста и др.).

Твоя задача:
1. Постараться восстановить корректный JSON по смыслу и структуре.
2. Привести его к строго заданному формату (см. ниже).
3. Если какие-то элементы повреждены — реконструируй их по контексту.
4. Если исходные данные неполные или в них **нет осмысленных трендов**, верни пустой список: `[]`.

## Формат вывода (JSON):
```json
[
  {
    "тренд": "Краткое утверждение, отражающее суть тренда (например: \"рост спроса на экологичные товары\")",
    "факт": "Цифры, статистика или цитата из текста, подтверждающая тренд (например: \"73% потребителей готовы платить больше за экологичную упаковку\")",
    "анализ": "Связный аналитический текст из 3–7 предложений, раскрывающий причины, влияние, последствия и прогноз тренда"
  }
]
'''

user_prompt = '''
Проанализируй и исправь следующий повреждённый JSON:

{json}
'''
fix_messages = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', user_prompt),
])


fix_chain = (
    fix_messages
    | llm 
    | JsonOutputParser()
)

In [ ]:
counter = 0
for doc in tqdm(docs):
    name = doc['name']
    tags = doc['tags']
    additional_text = doc['text']
    
    text_documents = []
    
    for pdf in doc['pdfs']:
        pdf_name = pdf['name']

        pdf_content = pdf.get('content', None)
        full_text = pdf.get('full_text', None)
        if full_text is None or len(full_text) > 0:
            counter += 1
        if full_text is None and pdf_content is not None:
            try:
                with tempfile.NamedTemporaryFile() as temp_file:
                    temp_file.write(pdf_content)
                    temp_file_path = temp_file.name
                    loader = PyPDFLoader(temp_file_path)
                    pdf_data = loader.load()
            except Exception as e:
                print(f"Error processing PDF {pdf_name}: {e}")
                pdf_data = None
            if pdf_data:
                full_text = "\n".join([doc.page_content for doc in pdf_data])
                pdf['full_text'] = full_text
        
        if full_text:
            if additional_text is None or len(additional_text) == 0:
                additional_text = 'нет краткого описание'
            text_document = f'**Документ с наименованием {pdf_name}, его содержимое:** {full_text}'
            text_documents.append(text_document)
    if len(text_documents) == 0:
        doc['summary'] = ''
    else:
        text_document = "\n".join(text_documents)
        trends = chain.invoke(
                {
                    'name': name,
                    'tags': ", ".join(tags), 
                    'additional_text': additional_text,
                    'text_document': text_document[:1_000 - 3] + '...',
                }
            )
        try:
            trends = json.loads(fix_json(trends))
        except:
            trends = fix_chain.invoke({'json': trends})
        doc['summary'] = trends

 96%|█████████▌| 95/99 [08:14<00:31,  7.81s/it]invalid pdf header: b'RIFF\x8c'
EOF marker not found


Error processing PDF Супертренд на здоровое питание — ключевая тенденция на рынке хлебобулочных и кондитерских изделий в России: Stream has ended unexpectedly


100%|██████████| 99/99 [08:25<00:00,  5.10s/it]


## Добавляем теги
Теги из kksb_client_profile

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Данные
data = {
    "date": "2023-10-15",
    "summary": "Саммари",
    "pdfs": [
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
        },
        {
            "full_text": "Полный текст и pdf, то что получилось вытащить из content",
        }
    ]
}

# llm = ChatOpenAI(model_name="gpt-4o-mini")

# Возможные теги
tags_list = [
    'Энергетика', 'Лизинг', 'Нефтегазовая промышленность', 'Пищевая промышленность',
    'Финансы', 'Органы государственного и муниципального управления', 'Материалы', 'Промышленность',
    'Телекоммуникации', 'Телекоммуникации и медиа', 'ИТ-технологии', 'ЖКХ', 'Услуги',
    'Электроэнергетика и ЖКХ', 'Строительные подрядчики', 'Сельское хозяйство', 'Товары первой необходимости',
    'Энергоносители', 'Товары выборочного спроса', 'Операции с недвижимым имуществом', 'Государство',
    'Недвижимость', 'Фармацевтика и здравоохранение', 'Химическая промышленность', 'Розничная торговля товарами первой необходимости',
    'Машиностроение', 'Металлургическая и горнодобывающая промышленность', 'Производство строительных материалов',
    'Розничная торговля товарами выборочного спроса', 'Финансовая деятельность', 'Легкая промышленность',
    'Транспорт и логистика', 'Производство потребительских товаров', 'Лесная и деревообрабатывающая и целлюлозно-бумажная промышленность', 'Туризм'
]
tags_list = [tag.lower() for tag in tags_list]

for data in tqdm(docs):
    text_sources = [
        'Отраслевые теги из документа: ',
        ', '.join([x for x in data["tags"] if x != 'Прочее']), 
        '\n Текст трендов: ', str(data["summary"]), 
    ]
    context_text = "\n".join(filter(None, text_sources))[:40_000]
    prompt = PromptTemplate(
        input_variables=["text", "tags"],
        template=(
            """Определи, какие из следующих тегов отрасли наиболее подходят для данного текста с отраслевыми трендами:
            ## Теги отрасли: 
            {tags}. 
            ## Тренды:
            {text}
            ## Формат ответа:
            Отвечай только списком релевантных трендам тегов через запятую, без пояснений. Пиши только те теги, которые релевантны трендам и есть в списке теги отрасли:{tags}. Не выдумывай другие теги и не изменя  формулировку этих тегов. Не пиши ничего кроме тегов. Если релевантных тегов нет не пиши ничего."""
        )
    )

    response = llm.invoke(prompt.format(text=context_text, tags=", ".join(tags_list))).content.strip()
    tags = [tag.strip() for tag in response.split(",") if tag.strip().lower() in tags_list]
    if len(tags) == 0:
        data["sber_tags"] = None
    else:
        data["sber_tags"] = ', '.join(list(set(tags)))

  0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 99/99 [01:27<00:00,  1.13it/s]


### Пример

# Сохраняемся

In [25]:
with open('docs.pkl', 'wb') as f:
    pickle.dump(docs, f)

In [26]:
def parse_date(date_str):
    # Пробуем разные форматы дат
    for fmt in ('%d.%m.%Y', '%Y-%m-%d', '%d-%m-%Y', '%Y.%m.%d', '%y-%m-%d', '%d/%m/%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except:
            continue
    return pd.NaT

In [36]:
df = pd.DataFrame(docs).rename(columns={'date': 'date_sum'})[['date_sum', 'sber_tags', 'summary']]
df['date_sum'] = df['date_sum'].apply(parse_date).dt.strftime('%d.%m.%Y')
df['sber_tags'] = df['sber_tags'].str.replace('\s*,\s*', ',', regex=True).str.split(',')
df = df.explode('sber_tags')
df['sber_tags'] = df['sber_tags'].str.strip()
df = df.reset_index().rename(columns={'index': 'row_id'})[['date_sum', 'sber_tags', 'summary', 'row_id']]
df = df.dropna(subset=['date_sum'], how='any')
df.to_excel('trends_with_dates.xlsx', index=False)